# **Projeto Quality Checker Kyndryl**
 
## **Made by Vìtor Ribeiro**

In [ ]:
# !apt-get install python3-distutils
!pip install python3-distutils
!pip install pyfiglet
!pip install pandas


In [2]:
import pandas as pd
import re
import json
from pyfiglet import Figlet
from distutils.util import strtobool
from datetime import datetime

f = Figlet(width=150)
print(f.renderText("Script made by\nVitor R. G. Gomes - Kyndryl\n2021-2022"))

 ____            _       _                         _        _           
/ ___|  ___ _ __(_)_ __ | |_   _ __ ___   __ _  __| | ___  | |__  _   _ 
\___ \ / __| '__| | '_ \| __| | '_ ` _ \ / _` |/ _` |/ _ \ | '_ \| | | |
 ___) | (__| |  | | |_) | |_  | | | | | | (_| | (_| |  __/ | |_) | |_| |
|____/ \___|_|  |_| .__/ \__| |_| |_| |_|\__,_|\__,_|\___| |_.__/ \__, |
                  |_|                                             |___/ 
__     ___ _               ____       ____      ____                                     _  __               _            _ 
\ \   / (_) |_ ___  _ __  |  _ \     / ___|    / ___| ___  _ __ ___   ___  ___          | |/ /   _ _ __   __| |_ __ _   _| |
 \ \ / /| | __/ _ \| '__| | |_) |   | |  _    | |  _ / _ \| '_ ` _ \ / _ \/ __|  _____  | ' / | | | '_ \ / _` | '__| | | | |
  \ V / | | || (_) | |    |  _ < _  | |_| |_  | |_| | (_) | | | | | |  __/\__ \ |_____| | . \ |_| | | | | (_| | |  | |_| | |
   \_/  |_|\__\___/|_|    |_| \_(_)  \____(_)  \____|\___/|_| 

In [ ]:
df = pd.read_excel("./incident_merged.xlsx")

pd.set_option("display.max_colwidth", None)
df.head()
df7 = df.groupby("Number")
len(df7)

In [ ]:
df2 = df[(df['Short description'].str.contains('autosys job',case=False)==False) & (df['Work notes'].str.contains('autosys job',case=False)==False)]
df3 = df2.groupby("Number")
len(df3)

In [ ]:
def checkCallList(entry):
    entry = entry.lower()
    if "call list:" and "job name:" and "instructions:" in entry:
        return True
    return False

In [ ]:
def checkTimeLine(entry):
    entry = entry.lower()
    timeLineDateArray = []
    workNotesTechArray = entry.split("\\n\\n")
    appendArray = False
    for pos in workNotesTechArray:
        if "email sent to" in pos and '@' in pos:
            match = re.search(
                "^([0-9]{4}-[0-9]{2}-[0-9]{2}\s[0-9]{2}:[0-9]{2}:[0-9]{2}(\.[0-9]{1,3})?)",
                pos,
            )
            if match:
                appendArray = True

        elif "call" in pos and re.search(
            "^(([0-9]{3}) |[0-9]{3}-)[0-9]{3}-[0-9]{4}$", pos
        ):
            match = re.search(
                "^([0-9]{4}-[0-9]{2}-[0-9]{2}\s[0-9]{2}:[0-9]{2}:[0-9]{2}(\.[0-9]{1,3})?)",
                pos,
            )
            if match:
                appendArray = True
        elif ("sent pager" or "pager sent") in pos and '@' in pos or re.search("^(([0-9]{3}) |[0-9]{3}-)[0-9]{3}-[0-9]{4}$", pos):
            match = re.search(
                "^([0-9]{4}-[0-9]{2}-[0-9]{2}\s[0-9]{2}:[0-9]{2}:[0-9]{2}(\.[0-9]{1,3})?)",
                pos,
            )
            if match:
                appendArray = True
        if appendArray:
            timeLineDateArray.append(
                datetime.strptime(match.group(0)[2:], "%y-%m-%d %H:%M:%S")
            )
        appendArray = False

    return timeLineDateArray

In [ ]:
def checkCloseNotes(entry):
    entry = entry.lower()
    array = re.split("\\\\n|:", entry)
    array = [i.strip(" ") for i in array]
    rcaTitle = ["RCA", "Root Cause", "rootcause"]
    rcaTitle = [element.lower() for element in rcaTitle]

    rcaDetail = [
        "Job Fail",
        "Job Failed",
        "Job Failure",
        "Job got faIled",
        "Autosys alarm",
        "Failed",
        "max run",
        "maxrun",
        "maxrunalarm",
        "generated due",
        "max run alarm",
        "max time",
        "maxtime",
        "abend",
        "ran longer",
        "killed",
        "unsuccessful",
        "job alert",
        "automated ritm",
        "ABENDED",
        "Job automation failure",
    ]
    rcaDetail = [element.lower() for element in rcaDetail]

    actionTitle = [
        "Action taken",
        "steps taken",
        "actiontaken",
        "stepstaken",
        "resolution",
        "actions taken",
    ]
    actionTitle = [element.lower() for element in actionTitle]

    actionTakenActions = [
        "Restarted",
        "force started",
        "force success",
        "force completed",
        "duplicated alert",
        "mark as success",
        "mark job success",
        "ON ICE",
        "ON HOLD",
        "CANCELLED",
        "complete",
        "completed",
        "success status",
        "kill",
        "killed",
    ]
    actionTakenActions = [element.lower() for element in actionTakenActions]

    actionTakenRequests = [
        "as per",
        "by",
        "with the comment",
        "in a new cycle",
        "RITM",
        "as requested by",
        "completed successfully",
        "per email",
        "INC",
        "asked to",
        "reset",
    ]
    actionTakenRequests = [element.lower() for element in actionTakenRequests]

    nextStepsTitle = [
        "Next Steps",
        "Future Steps",
        "Permanent Fix",
        "Next Step",
        "Further Action",
    ]
    nextStepsTitle = [element.lower() for element in nextStepsTitle]

    nextStepsDetail = ["None", "NA", "N/A"]
    nextStepsDetail = [element.lower() for element in nextStepsDetail]
    nextStepsDetail = "|".join(nextStepsDetail)
    if(len(array)<5):
        return False

    if (
        re.search("|".join(rcaTitle), array[0])
        and re.search("|".join(rcaDetail), array[1])
        and re.search("|".join(actionTitle), array[2])
        and re.search("|".join(actionTakenActions), array[3])
        and re.search("|".join(actionTakenRequests), array[3])
        and re.search("|".join(nextStepsTitle), array[4])
        and re.search("|".join(nextStepsDetail), array[5])
    ):
        if "completed successfully" in array[3] and not (
            rcaDetail == "max run"
            or rcaDetail == "maxrun"
            or rcaDetail == "maxrunalarm"
            or rcaDetail == "max run alarm"
        ):
            return False
        elif "INC" in array[3] and not (
            re.search("INC[0-9]{8,12}", array[3]) and "duplicated alert" in array[3]
        ):
            return False
        elif "RITM" in array[3] and not (re.search("RITM[0-9]{7}", array[3])):
            return False
        return True
    return False

In [ ]:
def checkEscalation(entry, group):
    if not entry:
        return False
    else:
        for i in range(group.shape[0]):
            df4 = group.iloc[[i]]
            if (
                df4["Value"].to_string(index=False).strip()
                == "Awaiting Technical Information"
            ):
                start = datetime.strptime(
                    df4["Start"].to_string(index=False).strip()[2:], "%y-%m-%d %H:%M:%S"
                )
                for date in entry:
                    if start > date and (start - date).seconds > 900:
                        return False
                    start = date
                return True

In [ ]:
employeesJson = json.load(open("./employees.json"))
cont=0
for key in df3.groups.keys():  
    df4 = df3.get_group(key).iloc[[df3.get_group(key).shape[0] - 1]]
    if df4.iloc[0]['Work notes']:
        assignedTo = df4["Assigned To"].to_string(index=False).strip()
        incidentNumber = df4["Number"].to_string(index=False).strip()
        print(cont,incidentNumber)
        made_sla = bool(strtobool(df4["Made SLA"].to_string(index=False).strip()))

        callList = checkCallList(df4["Work notes"].to_string(index=False).strip())
        timeLineCheck = checkTimeLine(df4["Work notes"].to_string(index=False).strip())
        closeNotes = checkCloseNotes(df4["Resolution Notes"].to_string(index=False).strip())

        # Para fazer a checagem de Escalation, é passado todas as timeLines encontradas

        # Entrada para forçar o erro durante a etapa de escalation
        #     timeLineErrada = []
        #     timeLineErrada.append(datetime.strptime("21-09-30 23:00:01", "%y-%m-%d %H:%M:%S"))
        #     escalation = checkEscalation(timeLineErrada,df2.get_group(key))

        # Entrada correta
        escalation = checkEscalation(timeLineCheck, df3.get_group(key))

        for idx, employee in enumerate(employeesJson):
            if assignedTo == employee["name"]:
                if made_sla:
                    employeesJson[idx]["totalSuccesses"] += 1
                else:
                    employeesJson[idx]["totalMistakes"] += 1
                    employeesJson[idx]["mistakesDetails"][0]["NumberOfMistakes"] += 1
                if callList:
                    employeesJson[idx]["totalSuccesses"] += 1
                else:
                    employeesJson[idx]["totalMistakes"] += 1
                    employeesJson[idx]["mistakesDetails"][1]["NumberOfMistakes"] += 1

                # Se não houver timeLine achada, já existe erro de escalation, pelo menos 1 timeline deve existir sempre
                if timeLineCheck:
                    employeesJson[idx]["totalSuccesses"] += 1
                    if escalation:
                        employeesJson[idx]["totalSuccesses"] += 1
                    else:
                        employeesJson[idx]["totalMistakes"] += 1
                        employeesJson[idx]["mistakesDetails"][2]["NumberOfMistakes"] += 1
                else:
                    # Itera erro de TimeLine
                    employeesJson[idx]["totalMistakes"] += 1
                    employeesJson[idx]["mistakesDetails"][3]["NumberOfMistakes"] += 1
                    # Itera erro de Escalation por tabela
                    employeesJson[idx]["totalMistakes"] += 1
                    employeesJson[idx]["mistakesDetails"][2]["NumberOfMistakes"] += 1
                if closeNotes:
                    employeesJson[idx]["totalSuccesses"] += 1
                else:
                    employeesJson[idx]["totalMistakes"] += 1
                    employeesJson[idx]["mistakesDetails"][4]["NumberOfMistakes"] += 1
                employeesJson[idx]["totalWorkedIncidents"] += 1

                cont+=1
        

jsonFinal = json.dumps(employeesJson, indent=2)
print(jsonFinal)

# **Análise Gráfica dos Resultados de cada operador**

In [ ]:
import matplotlib.pyplot as plt